In [1]:
import cmath as m
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
def refr_angle(n1, n2, aoi): # refraction angle 
    return np.arcsin(n1 * np.sin(aoi) / n2)

In [3]:
def refl_12_p(n1, n2, aoi): #reflection coef. of p-polar. between two surfaces 
    aor = refr_angle(n1, n2, aoi)
    #r_p
    return (n2*np.cos(aoi) - n1*np.cos(aor)) / (n2*np.cos(aoi) + n1*np.cos(aor))

In [4]:
def refl_12_s(n1, n2, aoi): #reflection coef. of s-polar. between two surfaces 
    aor = refr_angle(n1, n2, aoi)
    #r_s
    return (n1*np.cos(aoi) - n2*np.cos(aor)) / (n1*np.cos(aoi) + n2*np.cos(aor))

In [5]:
def tr_12_p(n1, n2, aoi): #transmission coef. of p-polar. between two surfaces 
    aor = refr_angle(n1, n2, aoi)
    #t_p
    return 2*n1*np.cos(aoi) / (n2*np.cos(aoi) + n1*np.cos(aor))

In [6]:
def tr_12_s(n1, n2, aoi): #transmission coef. of s-polar. between two surfaces 
    aor = refr_angle(n1, n2, aoi)
    #t_s
    return 2*n1*np.cos(aoi) / (n1*np.cos(aoi) + n2*np.cos(aor))

In [7]:
def refl_3(r12, r23, d2, n1, n2, lamda, phi): #reflection of triple interface
    psi = refr_angle(n1, n2, phi)
    phase = 2 * np.pi / lamda * n2 * np.cos(psi) * d2
    r123 = (r12 + r23 * np.exp(-2j * phase)) / (1 + r12 * r23 * np.exp(-2j * phase))
    return r123

In [8]:
#create massives

#division of angle
start = np.pi / 180 * 30
end = np.pi / 180 * 70
step = (end - start) / 1000 
aoi = np.arange(start, end, step)
#print(len(aoi))

#division of wavelenght
start, end = 450, 750
lamda = np.linspace(start, end, end-start+1, endpoint = True)


# optical constants of air
n_air = np.ones(end-start+1)

In [9]:
#read optical constants from file

with open('Silicon_ir.txt', 'r') as f_si, open('Dioxide_Silicon_ir.txt', 'r') as f_sio2, open('Gold.txt', 'r') as f_au, open('Bi2Te3_ox.txt') as f_bs, open('H2O.txt', 'r') as f_water:
    n_si = [complex(float(line.split()[1]),-float(line.split()[2])) for line in f_si.readlines() if  start <= int(line.split()[0]) <= end]
    n_si = np.array(n_si)
    #print(*n_si) 
    #print()
    n_sio2 = [complex(float(line.split()[1]),-float(line.split()[2])) for line in f_sio2.readlines() if  start <= int(line.split()[0]) <= end]
    n_sio2 = np.array(n_sio2)
    #print(*n_sio2) 
    n_au = [complex(float(line.split()[1]),-float(line.split()[2])) for line in f_au.readlines() if  start <= int(line.split()[0]) <= end]
    n_au = np.array(n_au)
    #print(*n_au)    
    n_bs = [complex(float(line.split()[1].replace(',','.')),-float(line.split()[2].replace(',','.'))) for line in f_bs.readlines() if start <= int(float(line.split()[0].replace(',','.'))) <= end]
    n_bs = np.array(n_bs)
    #print(*n_bs)
    #print(len(n_bs))
    n_w = [complex(float(line.split()[1]),-float(line.split()[2])) for line in f_water.readlines() if  start <= int(line.split()[0]) <= end]
    n_w = np.array(n_w)
    #print(len(n_w))

In [10]:
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

In [11]:
cs_r = CubicSpline(lamda[::2], n_bs.real)
cs_i = CubicSpline(lamda[::2], n_bs.imag)

In [12]:
n_bs_r = cs_r(lamda)
n_bs_i = cs_i(lamda)

In [14]:
n_bs = n_bs_r - 1j * n_bs_i

In [15]:
#need to vatiate lenght of sio2 and bi2se3
d_bs = [i for i in range(1, 5, 1)] 
d_sio2 = [i for i in range(250, 271, 5)]
for g in range(len(d_bs)):
    for h in range(len(d_sio2)):
        matrix = []
        for i in range(len(aoi)):

            #WATER - Bi2Se3 - SiO2 - Si p-polarization
            psi = refr_angle(n_w, n_bs, aoi[i])
            r12_p = refl_12_p(n_bs, n_sio2, psi)
            aoi_in_sio2 = refr_angle(n_w, n_sio2, aoi[i])
            r23_p = refl_12_p(n_sio2, n_si, aoi_in_sio2)
            r123_p = refl_3(r12_p, r23_p, d_sio2[h], n_bs, n_sio2, lamda, psi)
                #add layer with air
            r01_p = refl_12_p(n_w, n_bs, aoi[i])
            r0123_p = refl_3(r01_p, r123_p, d_bs[g], n_w, n_bs, lamda, aoi[i])
            matrix.append(abs(r0123_p))

#                 #Air - Bi2Se3 - SiO2 - Si s-polarization
#                     #Bi2Se3 - SiO2 - Si s-polarization
#                 psi = refr_angle(n_air[j], n_bs[j], aoi[i])
#                 r12_s = refl_12_s(n_bs[j], n_sio2[j], psi)
#                 aoi_in_sio2 = refr_angle(n_air[j], n_sio2[j], aoi[i])
#                 r23_s = refl_12_s(n_sio2[j], n_si[j], aoi_in_sio2)
#                 r123_s = refl_3(r12_s, r23_s, d_sio2[h], n_bs[j], n_sio2[j], lamda[j], psi)
#                     #add layer with air
#                 r01_s = refl_12_s(n_air[j], n_bs[j], aoi[i])
#                 r0123_s = refl_3(r01_s, r123_s, d_bs[g], n_air[j], n_bs[j], lamda[j], aoi[i])
#                 data.append(abs(r0123_s))


        #mini = min(min(matrix, key=min))
        from math import degrees
        mini = min(min(matrix, key=min))
        flag = 1
        for i in range(len(aoi)):
            for j in range(len(lamda)):
                if matrix[i][j] == mini:
                    print(d_bs[g], d_sio2[h], str(round(degrees(aoi[i]), 3)) + 'deg', str(lamda[j]) + 'nm', mini, sep=' : ')
                    flag = 0
                    break
            if flag == 0:
                break
    #print(*matrix) 

1 : 250 : 67.64deg : 450.0nm : 0.17070874517760493
1 : 255 : 68.28deg : 450.0nm : 0.15599456093402458
1 : 260 : 68.88deg : 450.0nm : 0.14094320182175493
1 : 265 : 69.48deg : 450.0nm : 0.12554288000328026
1 : 270 : 70.0deg : 450.0nm : 0.10980100301889238
2 : 250 : 67.44deg : 450.0nm : 0.1282713604801759
2 : 255 : 68.16deg : 450.0nm : 0.11243775382317166
2 : 260 : 68.84deg : 450.0nm : 0.09634299709837921
2 : 265 : 69.52deg : 450.0nm : 0.07995856304977746
2 : 270 : 70.0deg : 450.0nm : 0.06359878774491552
3 : 250 : 67.36deg : 450.0nm : 0.08281832758655992
3 : 255 : 68.12deg : 450.0nm : 0.06600648090847827
3 : 260 : 68.88deg : 450.0nm : 0.04897633112983949
3 : 265 : 69.6deg : 450.0nm : 0.03170839000469688
3 : 270 : 70.0deg : 452.0nm : 0.017131753768644856
4 : 250 : 67.32deg : 450.0nm : 0.03437579752779186
4 : 255 : 68.16deg : 450.0nm : 0.01672851753145385
4 : 260 : 68.8deg : 451.0nm : 0.0005351875407898025
4 : 265 : 68.04deg : 464.0nm : 0.0005528346482472127
4 : 270 : 67.36deg : 477.0nm : 0

In [16]:
%%time
d_bs = 5
d_sio2 = 260
#create massives

#division of angle
start = np.pi / 180 * 0
end = np.pi / 180 * 80
step = (end - start) / 4000 
aoi = np.arange(start, end, step)
#print(len(aoi))

#division of wavelenght
#lamda = l#list(range(350,751))

# optical constants of air
#n_air = [1 for i in range(601)]
#print(*aoi)
#print(*lamda)
matrix = []
for i in range(len(aoi)):

    #WATER - Bi2Se3 - SiO2 - Si p-polarization
    psi = refr_angle(n_w, n_bs, aoi[i])
    r12_p = refl_12_p(n_bs, n_sio2, psi)
    aoi_in_sio2 = refr_angle(n_w, n_sio2, aoi[i])
    r23_p = refl_12_p(n_sio2, n_si, aoi_in_sio2)
    r123_p = refl_3(r12_p, r23_p, d_sio2, n_bs, n_sio2, lamda, psi)
        #add layer with air
    r01_p = refl_12_p(n_w, n_bs, aoi[i])
    r0123_p = refl_3(r01_p, r123_p, d_bs, n_w, n_bs, lamda, aoi[i])
    matrix.append(np.angle(r0123_p))
#         data.append(m.phase(r0123_p)+m.pi)

CPU times: total: 3.81 s
Wall time: 3.85 s


In [ ]:
import plotly.graph_objects as go

structure = 'Water - Bi2Te3 - SiO2- Si'
polar = 'p'
fig = go.Figure(data=go.Heatmap(x=lamda,
                y=[degrees(c) for c in aoi],
                z=matrix,
                colorscale = "Hsv"))

fig.update_layout(
    title= f'Bi2Te3: {d_bs} nm, SiO2: {d_sio2} nm',
    xaxis_title="Wavelength, nm",
    yaxis_title="Angle of incidence, deg",
    legend_title="Phase",
    width=650,
    height=650,
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()